# Importing Necesary Libraries

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pickle
import os
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix,classification_report
from sklearn.model_selection import GridSearchCV
import pickle
from sklearn.externals import joblib

In [22]:
data = pd.read_excel('/content/datasets_33180_43520_heart.xlsx')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [23]:
# split the dataset into dependent('target') and independent features
# Feature, 'target' is the target variable or dependent feature in the data
x = data.drop('target', axis=1)
y = data['target']
x.shape, y.shape

((303, 13), (303,))

In [24]:
# Standard-Scaling the independent features
sc = StandardScaler()
X = sc.fit_transform(x)
# Using 80% of data for training and 20%, which is 1/5 of the total data for testing 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((242, 13), (61, 13), (242,), (61,))

In [25]:
# Logistic Regression after trying different techniques and algorithms. 
model = LogisticRegression()
model.fit(x_train,y_train)
pred = model.predict(x_test)

In [26]:
from sklearn.model_selection import cross_val_score
CV_Result = cross_val_score(model,x_train,y_train, cv=4, n_jobs=-1)
print(CV_Result)
print(CV_Result.mean())
print(CV_Result.std())

[0.83606557 0.85245902 0.81666667 0.8       ]
0.8262978142076502
0.019774678643025224


In [27]:
# Defining a function for printing some metrics
def metrics_info(x_test,pred):
    print('acccuracy score:', accuracy_score(y_test,pred))
    print('confusion matrix: ',confusion_matrix(y_test,pred))
    print('AUC-ROC Curve value :',roc_auc_score(y_test,pred))

metrics_info(x_test,pred)

acccuracy score: 0.8524590163934426
confusion matrix:  [[23  6]
 [ 3 29]]
AUC-ROC Curve value : 0.849676724137931


In [28]:
parameters = {'penalty':('l1', 'l2'), 'C':[1.0,10],'solver':('newton-cg','lbfgs','liblinear','sag','saga')}
clf = GridSearchCV(model,parameters)
clf.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver sag supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_

GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1.0, 10], 'penalty': ('l1', 'l2'),
                         'solver': ('newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [29]:
print('Best Penalty:', clf.best_estimator_.get_params()['penalty'])
print('Best C:', clf.best_estimator_.get_params()['C'])
print('Best Number Of Components:', clf.best_estimator_.get_params()['solver'])
print(clf.best_estimator_.get_params())

Best Penalty: l1
Best C: 1.0
Best Number Of Components: saga
{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l1', 'random_state': None, 'solver': 'saga', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [30]:
from sklearn.model_selection import cross_val_score
CV_Result = cross_val_score(clf,X,y, cv=4, n_jobs=-1)
print(CV_Result)
print(CV_Result.mean())
print(CV_Result.std())

[0.85526316 0.85526316 0.84210526 0.77333333]
0.8314912280701754
0.03400443999036261


In [31]:
inp = [('scale',StandardScaler()),('model',LogisticRegression(C= 1.0,penalty= 'l1',solver= 'saga',tol=0.0001))]
pipe = Pipeline(inp)
pipe.fit(x,y)

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('model',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l1', random_state=None,
                                    solver='saga', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [32]:
input = [59,1,0,135,234,0,1,161,0,0.5,1,0,3]    
pred = pipe.predict_proba([input])[0][1]

In [33]:
pred

0.3934236756957296

In [34]:
file=open('hd_model.pkl','wb')
pickle.dump(pipe,file)